## Video Action Recognition C3D
- Paper : (2014) https://arxiv.org/pdf/1412.0767.pdf
    - Learning Spatiotemporal Features with 3D Convolutional Networks
    
<hr>

## UCF 101 Dataset
- Action class : 101개
- 데이터 개수 : 13,320개 / Youtube
- class 당 clip 갯수 : 최소 100개 ~ 180개
- 크기 : (6.8G)
- 특징 
    - 다양한 camera motion과 object의 외형, 자세, viewpoint, 배경, illumination을 가짐
    - 다운로드 링크
        - https://www.crcv.ucf.edu/data/UCF101.php 
        
<hr>


## 1. Abstract & Introduction
- 2D convoltuion이 Spatiotemporal 정보에 대해 잘 파악하지 못하는, 특성을 보완하기 위해 제안된 3D convolution 구조
- 기존의 2D Conv 에서 3차원의 Conv을 이용하여 시공간적인 특징을 학습
- 3x3x3 conv kernel 사용 C3D Model 만듬
- 10개의 Layer만으로 UCF-101 52.8% 정확도 보임 (과거에는 더 낮았다고 함), 다른 벤치마크 성능들은 타 기술 대비 좋은 성능을 냄

![c3d_past_vs_current](img/c3d_past_vs_current.png)

- 지난 몇년간 이미지 conv를 통한 feature 학습에 많은 발전이 있었음 
     -> **But, Video Base에는 Motion modeling이 부족**
     -> **이를 3D Conv로 해결하고자 함**
     -> 3D Conv는 Feature extracting과 Motion을 동시에 Modeling 가능

## 2. 2D Conv -> 3D Conv Structure

![c3d_3dconv_structure](img/c3d_3dconv_structure.png)

- C3D는 3d conv와 3d pooling 덕에 temporal information을 더 잘 모델링 가능
- (a) 일반적인 2D conv
- (b) 복수의 이미지(Frame)에 2d적용 (다른 channel로 적용): 아웃풋은 image
- (c) 3d conv에서 아웃풋이 volume형태

![c3d_3dconv](img/c3d_3dconv.gif)

<hr>

## 3. Exploring 3D ConvNet
- 3D ConvNet 모델 설계를 위한 다양한 실험
- UCF101 데이터셋을 이용하여 학습 및 실험
- **Exploring kernel temporal depth**

![c3d_depth](img/c3d_depth.png)

- (1) 실험용 모델 정의
    - input dimension (Video Clip)
        - 3x16x128x171 (channel수, frame길이, height, width)
        - Training 때는 random crops (3x16×112×112) 로 사용

    - Conv Layer: 5개 (뒤에 pooling layer따라옴)
        - 각 Layer 필터 수: 64, 128, 256, 512, 512
        - **Kernel 사이즈 (d) : 실험적으로 여러 d 테스트**
        - stride 1

    - Pooling Layer: 5개
        - max pooling, kernel size: 2x2x2, stride 1
        - 첫번째 Pooling layer kernel size는 : 1x2x2 (temporal 신호를 일찍 합치지 않기 위해서)
    - FC Layer: 2개 
        - 2048 output
    - Softmax Loss Layer

- (2) 실험 방법
    - kernel temporal depth d를 주로 변경하며 실험
    - homogeneous temporal depth : 모든 conv layer가 같은 kernel temporal depth를 가짐
    - d : 1,3,5,7 실험 (depth-d로 명명)
        - depth-1는 각 프레임에 2d conv 적용한 것과 같음.
        - 증가(increasing) : 3-3-5-5-7
        - 감소(decreasing) : 7-5-5-3-3
    
- (3) 실험 결과
    - UCF101에서 2d conv(d=1)인 경우가 가장 성능이 나빴고, 3x3x3이 가장 좋은 성능을 내었다
    - UCF101은 중간규모였으니 이제 대규모에서 테스트

<hr>

## 4. C3D Model 
- Spatiotemporal feature learning

- Network Architecture (C3D Structure)
    - 8conv, 5pooling, 2FC, softmax

![c3d_model](img/c3d_model.png)


- Datasets
    - Sport-1M, 487 Sports categories, 
    - UCF 대비 클래스 5배, 데이터 100배
    - Sports-1M은 긴 동영상이 많기 때문에 무작위로 추출 ( 2 초 길이 클립 5 개 씩)
    
- Training
    - 128x171 -> 16x112x112 RESIZE, Croping
    - horizontally flip, SGD, Learning rate 0.003
    - I380K 데이터로 사전 훈련 된 모델에서 fine-tuning
    
- C3D는 무엇을 학습하는가?
![c3d_visualization](img/c3d_visualization.png)

- C3D는 초반 프레임에서는 외관에 초점, 점점 두드러지는 움직임을 추적
- 그림은 두가지 conv5의 feature map을 시각화
    - 1) 전체 사람(배경포함)에 초점 맞춘 후 나중엔 사람에게 초점
    - 2) 먼저 눈에 초점 맞춘 후 화장시 주위에 일어나는 동작을 초점
    - **따라서 C3D는 움직임과 외관 모두를 봄**

<hr>

## 5. Conclusion
- Contribution
    - 비디오 분석에서 3d conv가 시공간적 피쳐를 학습하는 것이 가능하며 Feature Extractor로서 3D Conv를 제안
    - 3d Conv에 가장 가장 최적인 termporal kernel length를 찾기 위해, 커널 사이즈와 아키텍쳐에 대한 전반적인 조사
    - 다른 비디오 분석 밴치마크에서 좋은 성능을 보임

- 문제점
    - long range temporal information에 대한 문제 (긴 시간에 대한 학습 잘 못함)
    - C3D의 계산량 문제

<hr>

## 참고문헌
- https://chacha95.github.io/2019-07-02-VideoUnderstanding2/
- https://22-22.tistory.com/74